In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pyrender
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from face_reconstruction.data.iphone import IPhoneDataLoader
from face_reconstruction.graphics import backproject_image, setup_standard_scene, get_perspective_camera
from env import IPHONE_DATASET_PATH

# 1. Setup Dataloader

In [ ]:
dataloader = IPhoneDataLoader()
img_width = dataloader.get_image_width()
img_height = dataloader.get_image_height()

In [ ]:
print(dataloader.get_frame_ids())

In [ ]:
frame_id = 0
frame = dataloader.get_frame(frame_id)

# 2. Color Image

In [ ]:
color_img = frame.get_color_image()
plt.imshow(color_img)
plt.show()

# 3. Depth Image

In [ ]:
depth_img = frame.get_depth_image()
plt.imshow(depth_img)
plt.colorbar()
plt.show()

# 4. Intrinsics

In [ ]:
intrinsics = frame.get_intrinsics()
print(intrinsics)

# 5. Match Color and Depth

In [ ]:
masked_color_img = np.array(color_img)
masked_color_img[depth_img > 1] = 0
plt.imshow(masked_color_img)
plt.show()

# 6 Project to 3D

In [ ]:
points = backproject_image(intrinsics, depth_img)
points = points[:, :3]
points[:,2] = -points[:, 2]  # Invert z-coordinate
points[:,1] = -points[:, 1]  # Invert y-coordinate
colors = color_img.reshape(-1, 3)  # Just flatten color image

In [ ]:
foreground_mask = depth_img.reshape(-1) < 1

In [ ]:
perspective_camera = get_perspective_camera(intrinsics, img_width, img_height)
scene = setup_standard_scene(perspective_camera)
scene.add(pyrender.Mesh.from_points(points[foreground_mask], colors=colors[foreground_mask]))

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True, viewport_size=(img_width, img_height))